# Introduction
In this notebook I will explore the words in the variable **item_description**. I will do this with the nice [WordCloud package](https://github.com/amueller/word_cloud) of **Andreas Mueller**.

These word clouds only serve to get a general feeling of what is in item_description. Nothing more than that. It can give a better understanding of its content and perhaps generate some ideas how to preprocess that variable or engineer other features from it. Hope you enjoy it!

## Loading the modules

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from wordcloud import WordCloud, STOPWORDS
import codecs
from scipy.misc import imread

## Loading the data

In [ ]:
df = pd.read_csv('../input/train.tsv', sep='\t')

## Creating a mask for the word clouds
As a mask I use the logo of Mercari. That makes things a bit more fun to look at. In order to do this you first need to base64-encode the image you want to use. Here, I've hidden that in order to keep the notebook clean.

In [ ]:
mercari_base64 = b'...'

In [ ]:
i = open("mercari_decoded.jpg", "wb")
i.write(codecs.decode(mercari_base64,'base64'))
i.close()
mercari_mask = imread("mercari_decoded.jpg")

In [ ]:
def create_wordcloud(subset):
    plt.figure(figsize=(20,15))
    wc = WordCloud(background_color="white", max_words=500, 
                   stopwords=STOPWORDS, max_font_size= 100, mask=mercari_mask)
    wc.generate(" ".join(df[subset]['item_description'].astype(str).str.lower()))
    plt.imshow(wc.recolor(colormap='ocean', random_state=37), alpha=0.98, interpolation="bilinear")
    plt.axis('off');

## Item Condition
Let's see if we notice specific words for the two extremes in **item_condition_id**. For that we create boolean mask to apply on the data frame and subset the data.

In [ ]:
poor_condition = df.item_condition_id == 5
good_condition = df.item_condition_id == 1

In [ ]:
create_wordcloud(poor_condition)

Relevant words for items in a poor condition: **used, broken, scratches, missing, cracked, ...**

In [ ]:
create_wordcloud(good_condition)

Relevant words for items in a good condition: **new, never used, new sealed, stainless, high quality, ...**

## Shipping costs
What are the words when **shipping** costs are included or not. The data description says:
> 1 if shipping fee is paid by seller and 0 by buyer

In [ ]:
shipping_excl = df.shipping == 1
shipping_incl = df.shipping == 0

In [ ]:
create_wordcloud(shipping_excl)

Obviously, **free shipping** is one of the most occurring words. Other words that pop up are: **free home, free, save shipping, ...**

In [ ]:
create_wordcloud(shipping_incl)

Weirdly enough **free shipping** is also used. Perhaps an error of the seller?

## Category
There are many different values for **category_name**. So we won't create a word cloud for all of them. It seems that there are different hierarchical levels in category_name, split by a slash symbol. 

In [ ]:
print('There are {} distinct category values'.format(len(df.category_name.unique())))

In [ ]:
makeup = df['category_name'].str.contains('Makeup', na = False)

In [ ]:
create_wordcloud(makeup)

Typical words for **Makeup** are: **lipstick, liquid, eye shadow, nail polish, lip gloss, sealed, never opened, ...** 

## Price
Finally, the most important variable in the data: **price**. Here I wanted to look at words used for the cheapest (bottom 5%) and most expensive (top 5%) items. Let's look at some descriptive statistics for price first.

In [ ]:
df.price.describe(percentiles=[.05, .95])

In [ ]:
bottom_price = df.price <= 6.0
top_price = df.price >= 75.0

In [ ]:
create_wordcloud(bottom_price)

We see **rm** coming up a few times. So here the seller probably wants to stress the low price.

In [ ]:
create_wordcloud(top_price)

Here we can find words like **louis vuitton, authentic, great condition, brand new, reasonable offer, ...** *rm* also comes up a few times, but not that often. The seller probably doesn't want to emphasize the higher price.

# Conclusion
That's it. These word clouds give us some insight in the typical words used for specific subsets of the training data. I hope you learned a bit from this notebook. If you have questions, feel free to leave a comment.